In [6]:
%load_ext autoreload
%autoreload 2
import os, sys, glob
import json
import re
import numpy as np
import pandas as pd
from natsort import natsorted

sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/utils/')
sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/utils/gentle')

import gentle
from config import *
import preproc_utils as preproc
# from preproc_utils import create_balanced_orders, get_consecutive_list_idxs, sort_consecutive_constraint, check_consecutive_spacing

# from text_utils import get_pos_tags, get_lemma

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set directories 

In [2]:
base_dir = '/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/'
stim_dir = os.path.join(base_dir, 'stimuli')
cache_dir = os.path.join('/dartfs/rc/lab/F/FinnLab/tommy/', 'models')

gentle_dir = os.path.join(stim_dir, 'gentle')

In [3]:
# def load_model_data(model_dir, model_name, task, window_size, top_n):
#     '''
#     Loads model data from directory
#     '''

#     model_dir = os.path.join(model_dir, task, model_name, f'window-size-{window_size}')
#     results_fn = natsorted(glob.glob(os.path.join(model_dir, f'*top-{top_n}*')))[0]

#     # load the data, remove nans
#     model_results = pd.read_csv(results_fn)
# #     model_results['glove_continuous_accuracy'] = model_results['glove_continuous_accuracy'].apply(np.nan_to_num)
# #     model_results['word2vec_continuous_accuracy'] = model_results['word2vec_continuous_accuracy'].apply(np.nan_to_num)

#     model_results['glove_avg_accuracy'] = model_results['glove_avg_accuracy'] #.apply(np.nan_to_num)
#     model_results['word2vec_avg_accuracy'] = model_results['word2vec_avg_accuracy'] #.apply(np.nan_to_num)
#     model_results['fasttext_avg_accuracy'] = model_results['fasttext_avg_accuracy'] #.apply(np.nan_to_num)

#     model_results['glove_max_accuracy'] = model_results['glove_max_accuracy'] #.apply(np.nan_to_num)
#     model_results['word2vec_max_accuracy'] = model_results['word2vec_max_accuracy'] #.apply(np.nan_to_num)
#     model_results['fasttext_max_accuracy'] = model_results['fasttext_max_accuracy'] #.apply(np.nan_to_num)

#     return model_results

def get_stim_candidate_idxs(task):
    '''
    Find the NWP candidate indices of a preprocessed transcript
    '''

    preproc_fn = os.path.join(STIM_DIR, 'preprocessed', task, f'{task}_transcript-preprocessed.csv')
    df_preproc = pd.read_csv(preproc_fn)
    nwp_idxs = np.where(df_preproc['NWP_Candidate'])[0]

    return df_preproc, nwp_idxs

# def divide_nwp_dataframe(df, accuracy_type, percentile):

#     df_divide = df.copy()

#     # first find the lowest and highest percentile for entropy
#     low_entropy_idxs = df['entropy'] < np.nanpercentile(df['entropy'], percentile)
#     high_entropy_idxs = df['entropy'] >= np.nanpercentile(df['entropy'], 100-percentile)

#     ## set names for entropy group
#     df_divide.loc[low_entropy_idxs, 'entropy_group'] = 'low'
#     df_divide.loc[high_entropy_idxs, 'entropy_group'] = 'high'

#     # repeat for continuous accuracy
#     low_accuracy_idxs = df[accuracy_type] < np.nanpercentile(df[accuracy_type], percentile)
#     high_accuracy_idxs = df[accuracy_type] >= np.nanpercentile(df[accuracy_type], 100-percentile)

#     ## set names for accuracy group
#     df_divide.loc[low_accuracy_idxs, 'accuracy_group'] = 'low'
#     df_divide.loc[high_accuracy_idxs, 'accuracy_group'] = 'high'

#     return df_divide #.dropna()

# # def get_quadrant_distributions(df_divide, indices):
    
# #     df_idx = df_divide.loc[indices]
    
# #     # get the items as a dictionary for passing out to aggregate
# #     quadrant_dist = {f'{labels[0]}-entropy_{labels[1]}-accuracy': round(len(df)/len(df_idx), 2) 
# #                  for labels, df in df_idx.groupby(['entropy_group', 'accuracy_group'])}

# #     df_quadrants = pd.DataFrame.from_dict(quadrant_dist, orient='index').T
    
# #     return df_quadrants

# def select_prediction_words(df_divide, remove_perc, select_perc, min_spacing_thresh=3):
#     '''
    
#     df_divide: candidate words divided into quartiles based on entropy and accuracy
    
#     remove_perc: percentage of words to remove based on proximity to other words
#         helps ensure decent spacing between presented words
        
#     select_perc: percentage of words to select for presentation    
    
#     '''
    
#     df_divide['spacing'] = np.hstack([np.nan, np.diff(df_divide.index)])
    
#     quadrant_distributions = get_quadrant_distributions(df_divide, df_divide.index).to_numpy()
    
#     updated = []

#     for i, df in df_divide.groupby(['entropy_group', 'accuracy_group']):
#         # find how many words to remove in the quadrant based on the percent
#         n_words = round(remove_perc * len(df))
#         df = df.sort_values(by='spacing').iloc[n_words:]
#         updated.append(df.sort_index())

#     updated = pd.concat(updated).sort_index()
#     updated_distributions = get_quadrant_distributions(updated, updated.index).to_numpy()
    
#     print (quadrant_distributions)
#     print (updated_distributions)
    
#     assert (np.isclose(quadrant_distributions, updated_distributions, atol=0.01).all())
    
#     # make sure it is scaled to the original dataframe
#     select_perc = select_perc/(1-remove_perc)
#     min_spacing = 0
#     RANDOM_STATE = 0
    
#     print (f'Selecting {select_perc*100:.2f}% of remaining items')
    
#     while (min_spacing < min_spacing_thresh):
#         # now sample the words from each quadrant
#         sampled = []

#         for i, df in updated.groupby(['entropy_group', 'accuracy_group']):

#             df_sampled = df.sample(frac=select_perc, random_state=RANDOM_STATE).sort_index()
#             sampled.append((len(df_sampled), df_sampled))

#         n_sampled, sampled = zip(*sampled)
#         sampled = pd.concat(sampled).sort_index()

#         min_spacing = np.diff(sampled.index).min()
        
#         RANDOM_STATE += 1
    
#     print (f'Min spacing of {min_spacing}')
#     print (f'{len(sampled)} total words')

#     return sampled

# import random

# def random_chunks(lst, n, shuffle=False):
#     """Created randomized n-sized chunks from lst."""
    
#     tmp_lst = lst.copy()
#     n_total = len(lst)
    
#     if shuffle:
#         random.shuffle(tmp_lst)
    
#     all_chunks = []
    
#     for i in range(0, len(tmp_lst), n):
#         all_chunks.append(tmp_lst[i:i + n])
    
#     # distribute remaining items across orders
#     if len(all_chunks) != n_total//n:
#         remainder = all_chunks.pop()
        
#         for i, item in enumerate(remainder):      
#             all_chunks[i%n].append(item)
    
#     # lastly sort for ordered indices
#     all_chunks = [sorted(chunk) for chunk in all_chunks]
    
#     return all_chunks
    

In [ ]:
adventuresinsayingyes
black
bronx
example_trial
eyespy
milkywayoriginal
milkywaysynonyms
milkywayvodka
nwp_practice_trial
piemanpni
prettymouth
shame
tunnel
wheretheressmoke

In [17]:
all_tasks_quadrants = []

models_dir = os.path.join(DERIVATIVES_DIR, 'model-predictions')
model_name = 'gpt2-xl'
task = 'black'

df_preproc, candidate_idxs = get_stim_candidate_idxs(task)

model_results = preproc.load_model_results(models_dir, model_name=model_name, task=task, top_n=5, window_size=100)
model_results.loc[:, 'binary_accuracy'] = model_results['binary_accuracy'].astype(bool)
model_results = model_results.iloc[candidate_idxs]

df_divide = preproc.divide_nwp_dataframe(model_results, accuracy_type='fasttext_avg_accuracy', percentile=45)


df_selected = preproc.select_prediction_words(df_divide, remove_perc=0.5, select_perc=0.4, min_spacing_thresh=2)

# percent_sampled = 0.
n_orders = 3
n_participants_per_item = 50
consecutive_spacing = 10

# find distribution of selected words from the divided quadrants
quadrant_distribution = preproc.get_quadrant_distributions(df_divide, df_selected.index).to_numpy()
deviation_threshold = 0.05
order_distributions = np.zeros((n_orders, 4))

# find indices for presentation and set number of items each subject sees
nwp_indices = sorted(df_selected.index)

# # Find lists with consecutive items violating our constraint

while not (np.allclose(quadrant_distribution, order_distributions, atol=deviation_threshold)):
    
    subject_experiment_orders = preproc.random_chunks(nwp_indices, len(nwp_indices)//n_orders, shuffle=True)
    
    print ('starting')
#     test_orders = subject_experiment_orders.copy()
    idxs = preproc.get_consecutive_list_idxs(subject_experiment_orders, consecutive_spacing=consecutive_spacing)
    pass_threshold, subject_experiment_orders = preproc.sort_consecutive_constraint(subject_experiment_orders, consecutive_spacing=consecutive_spacing, pass_threshold=50)
    
    if not pass_threshold:
        continue
    order_distributions = [preproc.get_quadrant_distributions(df_divide, order).to_numpy() for order in subject_experiment_orders]
    
    # sometimes the randomized order makes a quadrant be dropped --> reset and try again
    if not all([order.shape[-1] == 4 for order in order_distributions]):
        order_distributions = np.zeros((n_orders, 4))
# # Test again once we have completed resorting
# idxs = get_consecutive_list_idxs(subject_experiment_orders, consecutive_spacing=p.consecutive_spacing)
# print (f'Lists violating consecutive index constraint: {100*(len(idxs))/len(subject_experiment_orders)}%')

# uniq, counts = np.unique(subject_experiment_orders, return_counts=True)
# print (f'All counts per word: {np.sum(counts >= p.n_participants_per_item) / len(counts)*100}%')

# counts = Counter(tuple(o) for o in subject_experiment_orders)
# unique_orders = np.sum([v for k, v in counts.items()]) / len(counts)

# print (f'Unique orders: {unique_orders*100}%')

# orders_meeting_consecutive = np.sum([check_consecutive_spacing(order, consecutive_spacing=p.consecutive_spacing) for order in subject_experiment_orders]) / len(subject_experiment_orders)
# print (f'Consecutive constraint: {orders_meeting_consecutive*100}%'

/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/utils/preproc_utils.py:523: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'low' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_divide.loc[low_entropy_idxs, 'entropy_group'] = 'low'
/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/utils/preproc_utils.py:531: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'low' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_divide.loc[low_accuracy_idxs, 'accuracy_group'] = 'low'


[[0.17 0.33 0.34 0.16]]
[[0.17 0.33 0.34 0.16]]
Selecting 80.00% of remaining items
Tried random state: 0
Min spacing of 2
192 total words
starting
Starting pass #1
Number of lists w/ violation: 2
Starting pass #2
Number of lists w/ violation: 2
Starting pass #3
Number of lists w/ violation: 2
Starting pass #4
Number of lists w/ violation: 2
Starting pass #5
Number of lists w/ violation: 2
Starting pass #6
Number of lists w/ violation: 2
Starting pass #7
Number of lists w/ violation: 2
Starting pass #8
Number of lists w/ violation: 2
Starting pass #9
Number of lists w/ violation: 2
Starting pass #10
Number of lists w/ violation: 2
Starting pass #11
Number of lists w/ violation: 1
Starting pass #12
Number of lists w/ violation: 2
Starting pass #13
Number of lists w/ violation: 2
Starting pass #14
Number of lists w/ violation: 2
Starting pass #15
Number of lists w/ violation: 2
Starting pass #16
Number of lists w/ violation: 2
Starting pass #17
Number of lists w/ violation: 1
Starting pa

Number of lists w/ violation: 2
Starting pass #15
Number of lists w/ violation: 2
Starting pass #16
Number of lists w/ violation: 1
Starting pass #17
Number of lists w/ violation: 2
Starting pass #18
Number of lists w/ violation: 2
Starting pass #19
Number of lists w/ violation: 2
Starting pass #20
Number of lists w/ violation: 2
Starting pass #21
Number of lists w/ violation: 1
Starting pass #22
Number of lists w/ violation: 1
Starting pass #23
Number of lists w/ violation: 2
Starting pass #24
Number of lists w/ violation: 2
Starting pass #25
Number of lists w/ violation: 2
Starting pass #26
Number of lists w/ violation: 2
Starting pass #27
Number of lists w/ violation: 2
Starting pass #28
Number of lists w/ violation: 2
Starting pass #29
Number of lists w/ violation: 2
Starting pass #30
Number of lists w/ violation: 2
Starting pass #31
Number of lists w/ violation: 2
Starting pass #32
Number of lists w/ violation: 2
Starting pass #33
Number of lists w/ violation: 2
Starting pass #34


Number of lists w/ violation: 2
Starting pass #29
Number of lists w/ violation: 2
Starting pass #30
Number of lists w/ violation: 2
Starting pass #31
Number of lists w/ violation: 2
Starting pass #32
Number of lists w/ violation: 2
Starting pass #33
Number of lists w/ violation: 2
Starting pass #34
Number of lists w/ violation: 2
Starting pass #35
Number of lists w/ violation: 2
Starting pass #36
Number of lists w/ violation: 2
Starting pass #37
Number of lists w/ violation: 2
Starting pass #38
Number of lists w/ violation: 2
Starting pass #39
Number of lists w/ violation: 2
Starting pass #40
Number of lists w/ violation: 1
Starting pass #41
Number of lists w/ violation: 2
Starting pass #42
Number of lists w/ violation: 2
Starting pass #43
Number of lists w/ violation: 2
Starting pass #44
Number of lists w/ violation: 2
Starting pass #45
Number of lists w/ violation: 2
Starting pass #46
Number of lists w/ violation: 2
Starting pass #47
Number of lists w/ violation: 2
Starting pass #48


Number of lists w/ violation: 2
Starting pass #4
Number of lists w/ violation: 2
Starting pass #5
Number of lists w/ violation: 1
Starting pass #6
Number of lists w/ violation: 2
Starting pass #7
Number of lists w/ violation: 2
Starting pass #8
Number of lists w/ violation: 2
Starting pass #9
Number of lists w/ violation: 2
Starting pass #10
Number of lists w/ violation: 1
Starting pass #11
Number of lists w/ violation: 2
Starting pass #12
Number of lists w/ violation: 2
Starting pass #13
Number of lists w/ violation: 2
Starting pass #14
Number of lists w/ violation: 2
Starting pass #15
Number of lists w/ violation: 2
Starting pass #16
Number of lists w/ violation: 2
Starting pass #17
Number of lists w/ violation: 2
Starting pass #18
Number of lists w/ violation: 2
Starting pass #19
Number of lists w/ violation: 2
Starting pass #20
Number of lists w/ violation: 2
Starting pass #21
Number of lists w/ violation: 2
Starting pass #22
Number of lists w/ violation: 2
Starting pass #23
Number

...

Number of lists w/ violation: 2
Starting pass #2
Number of lists w/ violation: 2
Starting pass #3
Number of lists w/ violation: 2
Starting pass #4
Number of lists w/ violation: 2
Starting pass #5
Number of lists w/ violation: 2
Starting pass #6
Number of lists w/ violation: 2
Starting pass #7
Number of lists w/ violation: 2
Starting pass #8
Number of lists w/ violation: 2
Starting pass #9
Number of lists w/ violation: 2
Starting pass #10
Number of lists w/ violation: 2
Starting pass #11
Number of lists w/ violation: 1
Starting pass #12
Number of lists w/ violation: 1
Starting pass #13
Number of lists w/ violation: 2
Starting pass #14
Number of lists w/ violation: 2
Starting pass #15
Number of lists w/ violation: 2
Starting pass #16
Number of lists w/ violation: 2
Starting pass #17
Number of lists w/ violation: 2
Starting pass #18
Number of lists w/ violation: 2
Starting pass #19
Number of lists w/ violation: 2
Starting pass #20
Number of lists w/ violation: 2
Starting pass #21
Number o

Number of lists w/ violation: 1
Starting pass #22
Number of lists w/ violation: 2
Starting pass #23
Number of lists w/ violation: 2
Starting pass #24
Number of lists w/ violation: 2
Starting pass #25
Number of lists w/ violation: 1
Starting pass #26
Number of lists w/ violation: 2
Starting pass #27
Number of lists w/ violation: 1
Starting pass #28
Number of lists w/ violation: 2
Starting pass #29
Number of lists w/ violation: 2
Starting pass #30
Number of lists w/ violation: 2
Starting pass #31
Number of lists w/ violation: 2
Starting pass #32
Number of lists w/ violation: 2
Starting pass #33
Number of lists w/ violation: 2
Starting pass #34
Number of lists w/ violation: 2
Starting pass #35
Number of lists w/ violation: 2
Starting pass #36
Number of lists w/ violation: 1
Starting pass #37
Number of lists w/ violation: 2
Starting pass #38
Number of lists w/ violation: 2
Starting pass #39
Number of lists w/ violation: 2
Starting pass #40
Number of lists w/ violation: 2
Starting pass #41


Number of lists w/ violation: 2
Starting pass #2
Number of lists w/ violation: 2
Starting pass #3
Number of lists w/ violation: 2
Starting pass #4
Number of lists w/ violation: 2
Starting pass #5
Number of lists w/ violation: 2
Starting pass #6
Number of lists w/ violation: 2
Starting pass #7
Number of lists w/ violation: 2
Starting pass #8
Number of lists w/ violation: 2
Starting pass #9
Number of lists w/ violation: 2
Starting pass #10
Number of lists w/ violation: 2
Starting pass #11
Number of lists w/ violation: 1
Starting pass #12
Number of lists w/ violation: 2
Starting pass #13
Number of lists w/ violation: 2
Starting pass #14
Number of lists w/ violation: 2
Starting pass #15
Number of lists w/ violation: 2
Starting pass #16
Number of lists w/ violation: 2
Starting pass #17
Number of lists w/ violation: 2
Starting pass #18
Number of lists w/ violation: 2
Starting pass #19
Number of lists w/ violation: 1
Starting pass #20
Number of lists w/ violation: 2
Starting pass #21
Number o

Number of lists w/ violation: 2
Starting pass #23
Number of lists w/ violation: 2
Starting pass #24
Number of lists w/ violation: 2
Starting pass #25
Number of lists w/ violation: 2
Starting pass #26
Number of lists w/ violation: 2
Starting pass #27
Number of lists w/ violation: 2
Starting pass #28
Number of lists w/ violation: 2
Starting pass #29
Number of lists w/ violation: 2
Starting pass #30
Number of lists w/ violation: 2
Starting pass #31
Number of lists w/ violation: 2
Starting pass #32
Number of lists w/ violation: 2
Starting pass #33
Number of lists w/ violation: 1
Starting pass #34
Number of lists w/ violation: 2
Starting pass #35
Number of lists w/ violation: 2
Starting pass #36
Number of lists w/ violation: 2
Starting pass #37
Number of lists w/ violation: 1
Starting pass #38
Number of lists w/ violation: 2
Starting pass #39
Number of lists w/ violation: 2
Starting pass #40
Number of lists w/ violation: 2
Starting pass #41
Number of lists w/ violation: 1
Starting pass #42


Number of lists w/ violation: 2
Starting pass #50
Number of lists w/ violation: 2
starting
Starting pass #1
Number of lists w/ violation: 2
Starting pass #2
Number of lists w/ violation: 2
Starting pass #3
Number of lists w/ violation: 2
Starting pass #4
Number of lists w/ violation: 2
Starting pass #5
Number of lists w/ violation: 2
Starting pass #6
Number of lists w/ violation: 2
Starting pass #7
Number of lists w/ violation: 2
Starting pass #8
Number of lists w/ violation: 2
Starting pass #9
Number of lists w/ violation: 2
Starting pass #10
Number of lists w/ violation: 2
Starting pass #11
Number of lists w/ violation: 2
Starting pass #12
Number of lists w/ violation: 2
Starting pass #13
Number of lists w/ violation: 1
Starting pass #14
Number of lists w/ violation: 2
Starting pass #15
Number of lists w/ violation: 1
Starting pass #16
Number of lists w/ violation: 2
Starting pass #17
Number of lists w/ violation: 2
Starting pass #18
Number of lists w/ violation: 2
Starting pass #19


In [ ]:
N_ORDERS = 4
CONSECUTIVE_SPACING = 10 

# get baseline distribution of quadrants --> deviation threshold is the amount of error
# we tolerate from the distribution in each order
quadrant_distribution = get_quadrant_distributions(df_divide, df_selected.index).to_numpy()
deviation_threshold = 0.05
order_distributions = np.zeros((n_orders, 4))

# find indices for presentation and set number of items each subject sees
nwp_indices = sorted(df_selected.index)

# Find lists with consecutive items violating our constraint
while not (np.allclose(quadrant_distribution, order_distributions, atol=deviation_threshold)):

    # randomly chunk all indices into N_ORDERS
    subject_experiment_orders = random_chunks(nwp_indices, len(nwp_indices)//N_ORDERS, shuffle=True)
    subject_experiment_orders = sort_consecutive_constraint(subject_experiment_orders, consecutive_spacing=CONSECUTIVE_SPACING)
    
    # now find distribution of each order
    order_distributions = [get_quadrant_distributions(df_divide, order).to_numpy() for order in subject_experiment_orders]

    # sometimes the randomized order makes a quadrant be dropped --> reset and try again
    if not all([order.shape[-1] == 4 for order in order_distributions]):
        order_distributions = np.zeros((N_ORDERS, 4))

idxs = get_consecutive_list_idxs(subject_experiment_orders, consecutive_spacing=CONSECUTIVE_SPACING)
# now update df_preproc with our selected indices --> write out
selected_idxs = df_selected.index
df_final = df_preproc.copy()
df_final.loc[selected_idxs, ['entropy_group', 'accuracy_group']] = df_selected[['entropy_group', 'accuracy_group']]

df_final


# for each of the experiment orders
# for i, order in enumerate(subject_experiment_orders):

In [ ]:
test = [[12, 64, 81, 100, 295, 317, 341, 368, 380, 404, 431, 470, 483, 504, 517, 572, 598, 609, 666, 681, 704, 716, 741, 774, 907, 981, 1020, 1035, 1068, 1094, 1141, 1174, 1191, 1211, 1227, 1239, 1269, 1281, 1318, 1336, 1376, 1388, 1408, 1430, 1445, 1462, 1473, 1485, 1525], [19, 75, 148, 190, 235, 255, 305, 331, 354, 390, 409, 451, 476, 496, 507, 540, 558, 588, 626, 648, 670, 687, 710, 731, 813, 842, 856, 874, 919, 948, 994, 1009, 1043, 1055, 1112, 1136, 1182, 1233, 1246, 1273, 1300, 1311, 1325, 1380, 1394, 1416, 1436, 1493, 1516], [9, 49, 85, 116, 154, 201, 221, 244, 272, 302, 383, 401, 421, 446, 462, 528, 576, 673, 693, 707, 724, 747, 779, 799, 833, 845, 903, 914, 935, 970, 986, 1004, 1125, 1147, 1178, 1202, 1230, 1251, 1264, 1284, 1369, 1421, 1433, 1454, 1476, 1488, 1507, 1539], [6, 46, 90, 111, 143, 163, 212, 239, 263, 299, 346, 361, 373, 395, 417, 441, 500, 547, 567, 580, 603, 645, 663, 677, 713, 727, 744, 828, 848, 868, 941, 1017, 1062, 1075, 1100, 1133, 1187, 1217, 1236, 1291, 1303, 1328, 1343, 1358, 1412, 1440, 1479, 1496]]

In [ ]:
len(test)

In [ ]:
subject_experiment_orders

In [ ]:
np.tile(test, (50, 1)).shape[0]

In [ ]:
# get the items as a dictionary for passing out to aggregate
quadrant_dist = {f'{labels[0]}-entropy_{labels[1]}-accuracy': round(len(df)/len(df_divide), 2) 
             for labels, df in df_divide.groupby(['entropy_group', 'accuracy_group'])}

df_quadrants = pd.DataFrame.from_dict(quadrant_dist, orient='index').T

In [ ]:
order_distributions

In [ ]:
def create_n_random_orders(n_orders, n_participants_per_item, consecutive_spacing)

In [ ]:
get_quadrant_distributions(df_divide, order).to_numpy().min()

In [ ]:
test_orders

In [ ]:
order_distributions

In [ ]:
order_distributions

In [ ]:
np.diff(test_orders[0]).shape